<a href="https://colab.research.google.com/github/NMashalov/2023_OpenMipt_course/blob/main/lesson8/%D0%94%D0%97_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <a href="https://miptstats.github.io/courses/ad_mipt.html">Phystech@DataScience</a>
## Домашнее задание 8

**Правила, <font color="red">прочитайте внимательно</font>:**

* Выполненную работу нужно отправить телеграм-боту `@miptstats_pds_bot`. Для начала работы с ботом каждый раз отправляйте `/start`. **Работы, присланные иным способом, не принимаются.**
* Дедлайн см. в боте. После дедлайна работы не принимаются кроме случаев наличия уважительной причины.
* Прислать нужно ноутбук в формате `ipynb`.
* Выполнять задание необходимо полностью самостоятельно. **При обнаружении списывания все участники списывания будут сдавать устный зачет.**
* Решения, размещенные на каких-либо интернет-ресурсах, не принимаются. Кроме того, публикация решения в открытом доступе может быть приравнена к предоставлении возможности списать.
* Для выполнения задания используйте этот ноутбук в качестве основы, ничего не удаляя из него. Можно добавлять необходимое количество ячеек.
* Комментарии к решению пишите в markdown-ячейках.
* Выполнение задания (ход решения, выводы и пр.) должно быть осуществлено на русском языке.
* Если код будет не понятен проверяющему, оценка может быть снижена.
* Никакой код из данного задания при проверке запускаться не будет. *Если код студента не выполнен, недописан и т.д., то он не оценивается.*
* **Код из рассказанных на занятиях ноутбуков можно использовать без ограничений.**

**Правила оформления теоретических задач:**

* Решения необходимо прислать одним из следующих способов:
  * фотографией в правильной ориентации, где все четко видно, а почерк разборчив,
    * отправив ее как файл боту вместе с ноутбуком *или*
    * вставив ее в ноутбук посредством `Edit -> Insert Image` (<font color="red">фото, вставленные ссылкой, не принимаются</font>);
  * в виде $\LaTeX$ в markdown-ячейках.
* Решения не проверяются, если какое-то требование не выполнено. Особенно внимательно все проверьте в случае выбора второго пункта (вставки фото в ноутбук). <font color="red"><b>Неправильно вставленные фотографии могут не передаться при отправке.</b></font> Для проверки попробуйте переместить `ipynb` в другую папку и открыть его там.
* В решениях поясняйте, чем вы пользуетесь, хотя бы кратко. Например, если пользуетесь независимостью, то достаточно подписи вида "*X и Y незав.*"
* Решение, в котором есть только ответ, и отсутствуют вычисления, оценивается в 0 баллов.

**Баллы за задание:**

* Задача 1 &mdash;  40 баллов






-----

In [ ]:
# Bot check

# HW_ID: phds_hw8
# Бот проверит этот ID и предупредит, если случайно сдать что-то не то.

# Status: not final
# Перед отправкой в финальном решении удали "not" в строчке выше.
# Так бот проверит, что ты отправляешь финальную версию, а не промежуточную.
# Никакие значения в этой ячейке не влияют на факт сдачи работы.

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(font_scale=1.3)
%matplotlib inline

from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import train_test_split

## Задача 1

Попробуем поработать с категориальными признаками в задаче бинарной классификации.

В качестве данных возьмем следующиq датасет:

<https://www.kaggle.com/datasets/devzohaib/mushroom-edibility-classification?select=secondary_data.csv>. Будем предсказывать, является гриб съедобным или ядовитым.

#### Загрузка данных:

In [3]:
!pip install opendatasets

In [36]:
!pip install -U scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 89.9 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2


In [7]:
import opendatasets as od
od.download('https://www.kaggle.com/datasets/devzohaib/mushroom-edibility-classification/data')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: mashalovne
Your Kaggle Key: ··········


100%|██████████| 510k/510k [00:00<00:00, 104MB/s]

In [2]:
import pandas as pd
data = pd.read_csv('mushroom-edibility-classification/secondary_data.csv', sep=';').drop(['spore-print-color', 'gill-spacing', 'cap-surface', 'gill-attachment', 'stem-root', 'stem-surface', 'veil-type', 'veil-color'], axis=1).dropna()

In [4]:
from sklearn.model_selection import train_test_split
X = data.drop('class', axis=1)
y =  (data['class'] == 'p').astype('int')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

In [5]:
X_train.head(2)

,cap-diameter,cap-shape,cap-color,does-bruise-or-bleed,gill-color,stem-height,stem-width,stem-color,has-ring,ring-type,habitat,season
37692,2.00,p,n,f,g,6.64,2.66,y,t,e,m,a
54783,3.74,s,y,f,f,6.79,7.74,o,f,f,d,a


Найдем категориальные столбцы:

In [6]:
from sklearn.compose import make_column_selector as selector

categorical_columns_selector = selector(dtype_include=object)
categorical_columns = categorical_columns_selector(X_train)
categorical_columns

['cap-shape',
 'cap-color',
 'does-bruise-or-bleed',
 'gill-color',
 'stem-color',
 'has-ring',
 'ring-type',
 'habitat',
 'season']

In [9]:
cat_features = categorical_columns
numerical_features = X_train.columns.drop(cat_features).tolist()

Сравните качество получаемых моделей в двух случаях: первую модель обучите на данных без учета категориальных признаков, а для обучения второй модели каждый категориальный признак преобразуйте наиболее подходящим методом: OneHotEncoding'ом, LabelEncoding'ом или MeanEncoding'ом, объясните свой выбор. В качестве модели используйте случайный лес. Для оценки качества выберите подходящую метрику.

Не забывайте в процессе решения пояснять что вы делаете.

In [10]:
from sklearn.preprocessing import OneHotEncoder

from sklearn.feature_extraction import FeatureHasher

### Without cat features

In [11]:
from sklearn.ensemble import GradientBoostingClassifier

print('without cat features',
    GradientBoostingClassifier().fit(
        X_train.drop(cat_features,axis=1),
        y_train
    ).score(
        X_test.drop(
            cat_features,axis=1
        ),
    y_test)
)


without cat features 0.7063481228668942


### MeanEncoder

In [14]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, TargetEncoder
preprocessor = ColumnTransformer(
    [
        ("numerical", "passthrough", numerical_features),
        (
            "cat_feature",
            TargetEncoder(target_type="binary"),
            cat_features,
        ),

    ]
)

In [15]:
from sklearn.pipeline import make_pipeline

pipe = make_pipeline(
    preprocessor,
    GradientBoostingClassifier(),
)

print('',pipe.fit(X_train,y_train).score(X_test,y_test))


OHE 0.8667576791808874


## One Hot Encoder

In [16]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, TargetEncoder
preprocessor = ColumnTransformer(
    [
        ("numerical", "passthrough", numerical_features),
        (
            "cat_feature",
            OneHotEncoder(),
            cat_features,
        ),

    ]
)

In [17]:
from sklearn.pipeline import make_pipeline

pipe = make_pipeline(
    preprocessor,
    GradientBoostingClassifier(),
)

print('',pipe.fit(X_train,y_train).score(X_test,y_test))


 0.8736518771331058


## Catboost

In [18]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 2.5 MB/s eta 0:00:00


In [21]:
from catboost import CatBoostClassifier

print('CatBoost',CatBoostClassifier(cat_features=cat_features,verbose=False).fit(X_train,y_train).score(X_test,y_test))

CatBoost 0.9956313993174062


**Вывод:** <...>